# 小百合JobExpress板块信息
+ 储存到阿里云mysql中

In [1]:
from requests_html import HTMLSession
import regex as re
import string
import time
from mylog import MyLog as mylog
import pymysql
import pickle
from tqdm import tqdm

In [2]:
SUCCESSES = 0
ERRORS = 0

In [3]:
logger = mylog()
htmlsession = HTMLSession()
headers = {
    'user-agent':
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'accept':
    'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
    'referer':
    'http://bbs.nju.edu.cn/g/JobExpress'
}

In [4]:
start_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress'
pattern_index = re.compile('bbscon\?board=JobExpress&file=M\.(\d+)\.A&num=(\d+)')

In [5]:
keep_going = True
count_articles = 999999
url_articles = []

In [6]:
db = pymysql.connect("39.108.157.74","root","00genius00","JobExpress" )
cursor = db.cursor()

In [7]:
cursor.execute("SELECT MAX(id) FROM links")

1

In [8]:
count_crawled = cursor.fetchone()[0]
count_crawled

In [13]:
count_crawled -= 100
count_crawled

11914

In [14]:
def update_url_articles(res):
    global count_articles
    global keep_going
    global pattern_index
    for link in res.html.links:
        if link.startswith('bbscon?board=JobExpress&file=M.'):
            index = int(pattern_index.match(link).group(2))
            file = int(pattern_index.match(link).group(1))
            if index > count_crawled:
                count_articles = index if count_articles > index else count_articles
                url_articles.append((str(file), str(index), 'http://bbs.nju.edu.cn/'+link))
            else:
                keep_going = False

In [16]:
res = htmlsession.get(start_url, headers=headers)
update_url_articles(res)

In [17]:
while keep_going:
    count_articles = count_articles-20 if count_articles>=20 else 0
    target_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress&start=%s&type=doc' %str(count_articles)
    res = htmlsession.get(target_url, headers=headers)
    update_url_articles(res)
    time.sleep(0.5)

In [18]:
def insert(cursor, db, file, index, url):
    global SUCCESSES, ERRORS
    sql_query = "REPLACE INTO links VALUES ('%s', '%s', '%s')" %(str(file), str(index), url)
    try:
        cursor.execute(sql_query)
        db.commit()
        logger.info('成功: 插入id为: %s 的招聘信息' %str(index))
        SUCCESSES += 1
    except:
        db.rollback()
        logger.error('失败: 插入id为: %s 的招聘信息' %str(index))
        ERRORS += 1

In [21]:
for item in tqdm(url_articles):
    insert(cursor, db, item[0], item[1], item[2])

In [20]:
print(SUCCESSES, ERRORS)

40 0


# 第二部分: 爬具体文本

In [1]:
from requests_html import HTMLSession
import regex as re
import string
import time
from mylog import MyLog as mylog
import pymysql
import pickle
from tqdm import tqdm

In [5]:
file = 1535419630
id = 12000
url = 'http://bbs.nju.edu.cn/bbscon?board=JobExpress&file=M.1535419630.A&num=12000'

In [6]:
# SUCCESSES = 0
# ERRORS = 0

In [7]:
# logger = mylog()
# htmlsession = HTMLSession()
# headers = {
#     'user-agent':
#     'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
#     'accept':
#     'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
#     'referer':
#     'http://bbs.nju.edu.cn/g/JobExpress'
# }

In [8]:
# res = htmlsession.get(url, headers=headers)

In [15]:
# tags = res.html.find('td', first=True)

In [69]:
# pattern_user = re.compile('发信人: (.+?) \((.+?)\)')
# pattern_clicks = re.compile('本篇人气: (\d+)\n')
# pattern_title = re.compile('标  题: (.+?)\n')
# pattern_time = re.compile('发信站: 南京大学小百合站 \((.+?) (.+?) (\d+) (\d{2}:\d{2}:\d{2}) (\d+)\)\n\n')
# pattern_text = re.compile('发信站: 南京大学小百合站 \(.+?\)(.+?)--(.+?)来源')

In [26]:
# # user_id, user_name
# re_user = pattern_user.search(tags.full_text)
# user_id = re_user.group(1)
# user_name = re_user.group(2)
# print(user_id, user_name)

jxhnpl 孤之剑未尝不利


In [29]:
# # clicks
# re_clicks = pattern_clicks.search(tags.full_text)
# clicks = re_clicks.group(1)
# print(clicks)

58


In [32]:
# # title
# re_title = pattern_title.search(tags.full_text)
# title = re_title.group(1)
# print(title)

【汽车之家】商业产品/运营实习生


In [74]:
# # text
# re_text = pattern_text.search(tags.text)
# text = re_text.group(1)
# print(text)

 希望从事互联网行业，对商业化感兴趣的学弟学妹们不可错过~ 我们团队的实习生招聘，如果感兴趣，请邮件我简历~jiangxuehong@autohome.com.cn 岗位职责： 1、负责产品基础运营文档的维护更新； 2、承接业务部门广告投放需求，协助运营处理基础需求； 3、整理分析广告投放数据，完成日报周报等日常数据报告，协助运营同学把控广告效果； 4、协调内部资源，协助运营及产品进行线上问题处理跟进，促进产品迭代完善。 岗位要求： 1、本科及以上学历在读学生，可保证每周4个工作日及以上、连续4个月以上的实习时间； 2、对数据敏感，可以熟练操作excel、ppt等工具； 3、做事踏实认真，具备较强的快速学习能力、逻辑思维能力、执行力以及沟通协调能力； 4、有广告行业、商业化运营经验者优先。 


In [42]:
# # time
# re_time = pattern_time.search(tags.full_text)
# day, month, date, time, year = re_time.group(1), re_time.group(2), re_time.group(3), re_time.group(4), re_time.group(5)
# print(day, month, date, time, year)

Tue Aug 28 09:27:10 2018


In [82]:
print(file, id, url)

1535419630 12000 http://bbs.nju.edu.cn/bbscon?board=JobExpress&file=M.1535419630.A&num=12000


In [112]:
def get_details(url):
    res = htmlsession.get(url, headers=headers)
    tags = res.html.find('td', first=True)
    
    # user_id, user_name
    re_user = pattern_user.search(tags.full_text)
    user_id = re_user.group(1)
    user_name = re_user.group(2)

    # clicks
    re_clicks = pattern_clicks.search(tags.full_text)
    clicks = re_clicks.group(1)

    # title
    re_title = pattern_title.search(tags.full_text)
    title = re_title.group(1)

    # time
    try:
        re_time = pattern_time.search(tags.full_text)
        day, month, date, time, year = re_time.group(1), re_time.group(2), re_time.group(3), re_time.group(4), re_time.group(5)
    except:
        day, month, date, time, year = '', '', '', '', '' 

    # text
    try:
        re_text = pattern_text.search(tags.text)
        text = re_text.group(1)
    except:
        text = ''
    
    return user_id, user_name, clicks, title, day, month, date, time, year, text

In [119]:
def main():
    global SUCCESSES
    global ERRORS

    logger = mylog()
    htmlsession = HTMLSession()
    headers = {
        'user-agent':
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
        'accept':
        'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
        'referer':
        'http://bbs.nju.edu.cn/g/JobExpress'
    }
    
    pattern_user = re.compile('发信人: (.+?) \((.+?)\)')
    pattern_clicks = re.compile('本篇人气: (\d+)\n')
    pattern_title = re.compile('标  题: (.+?)\n')
    pattern_time = re.compile('发信站: 南京大学小百合站 \((.+?) (.+?) (\d+) (\d{2}:\d{2}:\d{2}) (\d+)\)\n\n')
    pattern_text = re.compile('发信站: 南京大学小百合站 \(.+?\)(.+?)--(.+?)来源')
    
    db = pymysql.connect("39.108.157.74","root","00genius00","JobExpress" )
    cursor = db.cursor()
    
    query = 'SELECT * FROM links WHERE crawled = 0 order by id desc limit 100;'
    data = pd.read_sql_query(query, db)
    data = pd.DataFrame(data)
    
    for i in tqdm(range(len(data))):
        try:
            file, id, url = data.loc[i, ['file', 'id', 'links']]
            user_id, user_name, clicks, title, day, month, date, time, year, text = get_details(url)

            sql_query = "REPLACE INTO details VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" %(str(file), str(id), url, user_id, user_name, clicks, title, day, month, date, time, year, text)

            try:
                cursor.execute(sql_query)
                cursor.execute('UPDATE links SET crawled = 1 WHERE file = %s and id = %s' %(file, id))
                db.commit()
                logger.info('成功: 爬取&插入id为: %s 的招聘信息' %str(id))
                SUCCESSES += 1
            except:
                db.rollback()
                logger.error('失败: 插入id为: %s 的招聘信息' %str(id))
                ERRORS += 1
        except:
            logger.error('失败: 爬取id为: %s 的招聘信息' %str(id))
    
    db.close()

In [120]:
main()

  0%|                                                                          | 0/100 [00:00<?, ?it/s]2018-08-29 01:16:47,060 ERROR    ydzhang    失败: 爬取id为: 12018 的招聘信息

2018-08-29 01:16:47,069 ERROR    ydzhang    失败: 爬取id为: 12017 的招聘信息

2018-08-29 01:16:47,191 INFO     ydzhang    成功: 爬取&插入id为: 12012 的招聘信息

  3%|█▉                                                                | 3/100 [00:00<00:04, 21.57it/s]2018-08-29 01:16:47,367 INFO     ydzhang    成功: 爬取&插入id为: 12011 的招聘信息

  4%|██▋                                                               | 4/100 [00:00<00:07, 12.68it/s]2018-08-29 01:16:47,493 INFO     ydzhang    成功: 爬取&插入id为: 12010 的招聘信息

  5%|███▎                                                              | 5/100 [00:00<00:08, 11.33it/s]2018-08-29 01:16:47,617 INFO     ydzhang    成功: 爬取&插入id为: 12009 的招聘信息

  6%|███▉                                                              | 6/100 [00:00<00:08, 10.61it/s]2018-08-29 01:16:47,745 INFO     ydzhang    成功: 爬取&插入id为: 12008 的

In [84]:
import pandas as pd

In [87]:
db = pymysql.connect("39.108.157.74","root","00genius00","JobExpress" )
cursor = db.cursor()
query = 'SELECT * FROM links WHERE crawled = 0;'
data = pd.read_sql_query(query, db)
data = pd.DataFrame(data)

In [101]:
data.loc[0, ['file', 'id', 'links']]

file                                            1286976702
id                                                       0
links    http://bbs.nju.edu.cn/bbscon?board=JobExpress&...
Name: 0, dtype: object

In [99]:
a

file                                              1286976702
id                                                         0
links      http://bbs.nju.edu.cn/bbscon?board=JobExpress&...
crawled                                                    0
Name: 0, dtype: object

In [ ]:
for i in tqdm(range(len(data))):
    file, id, url = data.loc[i, ['file', 'id', 'links']]
    user_id, user_name, clicks, title, day, month, date, time, year, text = get_details(url)
    
    sql_query = "REPLACE INTO details VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" %(str(file), str(index), url, user_id, user_name, clicks, title, day, month, date, time, year, text)
    
    try:
        cursor.execute(sql_query)
        db.commit()
        logger.info('成功: 插入id为: %s 的招聘信息' %str(index))
        SUCCESSES += 1
    except:
        db.rollback()
        logger.error('失败: 插入id为: %s 的招聘信息' %str(index))
        ERRORS += 1

In [81]:
'%s %s %s %s %s %s %s %s %s %s ' %get_details(url)

'jxhnpl 孤之剑未尝不利 62 【汽车之家】商业产品/运营实习生 Tue Aug 28 09:27:10 2018  希望从事互联网行业，对商业化感兴趣的学弟学妹们不可错过~ 我们团队的实习生招聘，如果感兴趣，请邮件我简历~jiangxuehong@autohome.com.cn 岗位职责： 1、负责产品基础运营文档的维护更新； 2、承接业务部门广告投放需求，协助运营处理基础需求； 3、整理分析广告投放数据，完成日报周报等日常数据报告，协助运营同学把控广告效果； 4、协调内部资源，协助运营及产品进行线上问题处理跟进，促进产品迭代完善。 岗位要求： 1、本科及以上学历在读学生，可保证每周4个工作日及以上、连续4个月以上的实习时间； 2、对数据敏感，可以熟练操作excel、ppt等工具； 3、做事踏实认真，具备较强的快速学习能力、逻辑思维能力、执行力以及沟通协调能力； 4、有广告行业、商业化运营经验者优先。  '